In [88]:
import requests, re
from bs4 import BeautifulSoup as bs
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time, random, pickle, difflib
import Levenshtein as lev
from urllib.request import urlopen
import urllib.parse, os
import numpy as np
from IPython.display import display

Merge 3 dataframes.

In [14]:
commuter_df = pd.read_pickle('Figures/commuter_df').dropna()

In [21]:
commuter_df = commuter_df.loc[1:]
commuter_df

,Rank,Area Name,St1,Size,St2,PlcCode,Population,WorkersWP,WorkersLP,Daytime,Daytime1,%Daytime1,WorkersLW,%WorkersLW
1,1,New York,NY,1,36,51000,"8,008,278","3,755,130","3,192,070","8,571,338","563,060",7.0,"2,922,206",91.5
2,2,Los Angeles,CA,1,06,44000,"3,694,820","1,622,772","1,494,895","3,822,697","127,877",3.5,"943,489",63.1
3,3,Chicago,IL,1,17,14000,"2,896,016","1,334,467","1,192,139","3,038,344","142,328",4.9,"841,329",70.6
4,4,Houston,TX,1,48,35000,"1,953,631","1,244,999","841,686","2,356,944","403,313",20.6,"681,785",81.0
5,5,Philadelphia,PA,1,42,60000,"1,517,550","659,991","569,761","1,607,780","90,230",5.9,"429,608",75.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8684,6470,Meridian CDP,CO,9,08,50012,184,"7,905",112,"7,977","7,793",4235.3,10,8.9
8685,6471,Rock Island Arsenal CDP,IL,9,17,65104,145,"3,941",71,"4,015","3,870",2669.0,25,35.2
8686,6472,Black Hawk,CO,9,08,07025,118,"2,879",88,"2,909","2,791",2365.3,29,33.0
8687,6473,Vernon,CA,9,06,82422,91,"37,472",36,"37,527","37,436",41138.5,17,47.2


In [147]:
city_zip_df = pd.read_csv('Figures/zips.csv', index_col=0).dropna()
city_zip_df

,City,State,Zip Code
1.0,NEW YORK,NY,10001.0
2.0,NEW YORK,NY,10002.0
3.0,NEW YORK,NY,10003.0
4.0,NEW YORK,NY,10004.0
5.0,NEW YORK,NY,10005.0
...,...,...,...
NaN,JERSEY CITY,NJ,7308.0
NaN,JERSEY CITY,NJ,7310.0
NaN,JERSEY CITY,NJ,7311.0
NaN,JERSEY CITY,NJ,7395.0


In [178]:
demog_info = pd.read_pickle('Figures/zip_demog_info')
demog_info

,Population,Population Density,Median Household Income,Median Age:,"Car, truck, or van",Less than High School Diploma,High School Graduate,Associate's degree,Bachelor's degree,Master's degree,Professional school degree,Doctorate degree,Latitude,Longitude
10001,21102,33959,"$81,671",35,6%,8%,19%,3%,38%,21%,7%,3%,40.75,-73.99
10002,81410,92573,"$33,218",40,10.8%,36%,31%,4%,21%,6%,2%,1%,40.72,-73.99
10003,56024,97188,"$92,540",32,5.2%,4%,16%,2%,43%,21%,9%,4%,40.73,-73.99
10004,3089,5519,"$129,313",32,7%,2%,16%,4%,42%,22%,13%,2%,40.70,-74.02
10005,7135,97048,"$124,670",29,3%,2.3%,8.3%,0.7%,47.8%,23.9%,12.9%,4.1%,40.705,-74.005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
07308,None,None,None,None,None,None,None,None,None,None,None,None,40.717,-74.043
07310,12838,20880,"$133,636",30,11.4%,3%,11%,1%,33%,42%,4%,6%,40.73,-74.04
07311,522,17954,"$219,643",33,11%,0%,4%,0%,28%,23%,21%,25%,40.719,-74.034
07395,None,None,None,None,None,None,None,None,None,None,None,None,40.717,-74.043


In [179]:
commuter_df_NYNJCTMD = pd.read_pickle('Figures/commuter_df_NYNJCTMD')
commuter_df_NYNJCTMD = commuter_df_NYNJCTMD[['Area Name', 'St1', 'WorkersLP', 'WorkersWP']]
commuter_df_NYNJCTMD
# commuter_df_NYNJCTMD['ER Ratio'] = commuter_df_NYNJCTMD.WorkersWP / commuter_df_NYNJCTMD.WorkersLP

,Area Name,St1,WorkersLP,WorkersWP
0,New York,NY,"3,192,070","3,755,130"
16,Baltimore,MD,"249,373","341,998"
57,Buffalo,NY,"110,640","158,245"
62,Newark,NJ,"87,720","147,395"
71,Jersey,NJ,"100,750","93,622"
...,...,...,...,...
6427,East Hampton,NY,615,"5,127"
6444,East Garden CDP,NY,485,"23,591"
6453,Wrightstown borough,NJ,349,"6,698"
6458,Upper Marlboro,MD,339,"5,565"


Convert string numbers to floats and create ER column.

In [180]:
dict_columns_type = {'WorkersLP': int, 'WorkersWP': int}
commuter_df_NYNJCTMD = commuter_df_NYNJCTMD.apply(lambda x: x.replace({',':''}, regex=True)).astype(dict_columns_type)
commuter_df_NYNJCTMD['ER Ratio'] = commuter_df_NYNJCTMD['WorkersWP'] / commuter_df_NYNJCTMD['WorkersLP']
commuter_df_NYNJCTMD = commuter_df_NYNJCTMD.replace({'borough':'', 'CDP':''}, regex=True)
commuter_df_NYNJCTMD.loc[71, 'Area Name'] = 'Jersey City'
commuter_df_NYNJCTMD

,Area Name,St1,WorkersLP,WorkersWP,ER Ratio
0,New York,NY,3192070,3755130,1.176393
16,Baltimore,MD,249373,341998,1.371432
57,Buffalo,NY,110640,158245,1.430269
62,Newark,NJ,87720,147395,1.680290
71,Jersey City,NJ,100750,93622,0.929251
...,...,...,...,...,...
6427,East Hampton,NY,615,5127,8.336585
6444,East Garden,NY,485,23591,48.641237
6453,Wrightstown,NJ,349,6698,19.191977
6458,Upper Marlboro,MD,339,5565,16.415929


Add zipcodes to city columns using Levenshtein distance  

In [181]:
city_zip_df.tail()

,City,State,Zip Code
NaN,JERSEY CITY,NJ,7308.0
NaN,JERSEY CITY,NJ,7310.0
NaN,JERSEY CITY,NJ,7311.0
NaN,JERSEY CITY,NJ,7395.0
NaN,JERSEY CITY,NJ,7399.0


In [182]:
unique_city_names_zip_df = list(city_zip_df.City.unique())
# unique_city_names_zip_df

In [183]:
def match_city(x):
    try: return difflib.get_close_matches(x.upper(), unique_city_names_zip_df, cutoff=0.9)[0]
    except IndexError: return np.nan

In [184]:
commuter_df_NYNJCTMD['Area Name'] = commuter_df_NYNJCTMD['Area Name'].map(match_city)
commuter_df_NYNJCTMD.head()

,Area Name,St1,WorkersLP,WorkersWP,ER Ratio
0,NEW YORK,NY,3192070,3755130,1.176393
16,BALTIMORE,MD,249373,341998,1.371432
57,BUFFALO,NY,110640,158245,1.430269
62,NEWARK,NJ,87720,147395,1.680290
71,JERSEY CITY,NJ,100750,93622,0.929251


In [185]:
commuter_zip = commuter_df_NYNJCTMD.merge(city_zip_df, left_on='Area Name', right_on='City', how='left')
commuter_zip

,Area Name,St1,WorkersLP,WorkersWP,ER Ratio,City,State,Zip Code
0,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10001.0
1,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10002.0
2,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10003.0
3,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10004.0
4,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10005.0
...,...,...,...,...,...,...,...,...
1846,UPPER MARLBORO,MD,339,5565,16.415929,UPPER MARLBORO,MD,20774.0
1847,UPPER MARLBORO,MD,339,5565,16.415929,UPPER MARLBORO,MD,20775.0
1848,UPPER MARLBORO,MD,339,5565,16.415929,UPPER MARLBORO,MD,20792.0
1849,TETERBORO,NJ,13,6697,515.153846,TETERBORO,NJ,7608.0


In [187]:
commuter_zip['Zip Code'] = commuter_zip['Zip Code'].map(clean_zips)

In [188]:
commuter_zip.head()

,Area Name,St1,WorkersLP,WorkersWP,ER Ratio,City,State,Zip Code
0,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10001
1,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10002
2,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10003
3,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10004
4,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10005


In [189]:
commuter_zip_demog = commuter_zip.merge(demog_info, left_on='Zip Code', right_index=True, how='left')
commuter_zip_demog.head()

,Area Name,St1,WorkersLP,WorkersWP,ER Ratio,City,State,Zip Code,Population,Population Density,...,"Car, truck, or van",Less than High School Diploma,High School Graduate,Associate's degree,Bachelor's degree,Master's degree,Professional school degree,Doctorate degree,Latitude,Longitude
0,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10001,21102,33959,...,6%,8%,19%,3%,38%,21%,7%,3%,40.75,-73.99
1,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10002,81410,92573,...,10.8%,36%,31%,4%,21%,6%,2%,1%,40.72,-73.99
2,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10003,56024,97188,...,5.2%,4%,16%,2%,43%,21%,9%,4%,40.73,-73.99
3,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10004,3089,5519,...,7%,2%,16%,4%,42%,22%,13%,2%,40.70,-74.02
4,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10005,7135,97048,...,3%,2.3%,8.3%,0.7%,47.8%,23.9%,12.9%,4.1%,40.705,-74.005


In [190]:
merged_EV_stations_df = pd.read_pickle('Figures/merged_EV_stations_df')
commuter_zip_demog_EV = commuter_zip_demog.merge(merged_EV_stations_df, left_on = 'Zip Code', right_index=True, how='left')
commuter_zip_demog_EV

,Area Name,St1,WorkersLP,WorkersWP,ER Ratio,City,State,Zip Code,Population,Population Density,...,High School Graduate,Associate's degree,Bachelor's degree,Master's degree,Professional school degree,Doctorate degree,Latitude,Longitude,reg_vehicles,num_stations
0,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10001,21102,33959,...,19%,3%,38%,21%,7%,3%,40.75,-73.99,495.0,10.0
1,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10002,81410,92573,...,31%,4%,21%,6%,2%,1%,40.72,-73.99,454.0,6.0
2,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10003,56024,97188,...,16%,2%,43%,21%,9%,4%,40.73,-73.99,1210.0,14.0
3,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10004,3089,5519,...,16%,4%,42%,22%,13%,2%,40.70,-74.02,118.0,1.0
4,NEW YORK,NY,3192070,3755130,1.176393,NEW YORK,NY,10005,7135,97048,...,8.3%,0.7%,47.8%,23.9%,12.9%,4.1%,40.705,-74.005,219.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1846,UPPER MARLBORO,MD,339,5565,16.415929,UPPER MARLBORO,MD,20774,43013,1238,...,47%,7%,23%,14%,2%,1%,38.89,-76.77,2713.0,9.0
1847,UPPER MARLBORO,MD,339,5565,16.415929,UPPER MARLBORO,MD,20775,None,None,...,None,None,None,None,None,None,38.8,-76.8,NaN,NaN
1848,UPPER MARLBORO,MD,339,5565,16.415929,UPPER MARLBORO,MD,20792,None,None,...,None,None,None,None,None,None,38.8,-76.8,NaN,NaN
1849,TETERBORO,NJ,13,6697,515.153846,TETERBORO,NJ,07608,67,43,...,51%,0%,30%,0%,0%,0%,40.85,-74.06,2.0,0.0


In [191]:
commuter_zip_demog_EV.dropna().to_csv('Figures/commuter_zip_demog_EV.csv')

The list with daily commuters is missing a lot of zip codes so we need to redo the merge but omit the daily commuter columns. This will decrease the number of features but increase the total number of data points.

In [206]:
def clean_zips(zipp):
    zipp = str(zipp)
    if len(zipp) < 5:
        zipp = '0' + zipp
    if len(zipp) < 4:
        zipp = '0' + zipp
    return zipp

Fill NaN with 0's

In [212]:
city_zip_df = pd.read_csv('Figures/zips.csv', index_col=0).dropna()
city_zip_df['Zip Code'] = city_zip_df['Zip Code'].apply(lambda x: clean_zips(x))
city_zip_df = city_zip_df.merge(merged_EV_stations_df, left_on='Zip Code', right_index=True, how='left')
city_zip_df.fillna(0, inplace=True)
city_zip_df

,City,State,Zip Code,reg_vehicles,num_stations
1,NEW YORK,NY,10001,495.0,10.0
2,NEW YORK,NY,10002,454.0,6.0
3,NEW YORK,NY,10003,1210.0,14.0
4,NEW YORK,NY,10004,118.0,1.0
5,NEW YORK,NY,10005,219.0,3.0
...,...,...,...,...,...
2199,Wayne,NY,14893,0.0,0.0
2200,Wellsburg,NY,14894,7.0,0.0
2202,Whitesville,NY,14897,21.0,0.0
2203,Woodhull,NY,14898,0.0,0.0


In [213]:
demog_info = pd.read_pickle('Figures/zip_demog_info')
demog_info

,Population,Population Density,Median Household Income,Median Age:,"Car, truck, or van",Less than High School Diploma,High School Graduate,Associate's degree,Bachelor's degree,Master's degree,Professional school degree,Doctorate degree,Latitude,Longitude
10001,21102,33959,"$81,671",35,6%,8%,19%,3%,38%,21%,7%,3%,40.75,-73.99
10002,81410,92573,"$33,218",40,10.8%,36%,31%,4%,21%,6%,2%,1%,40.72,-73.99
10003,56024,97188,"$92,540",32,5.2%,4%,16%,2%,43%,21%,9%,4%,40.73,-73.99
10004,3089,5519,"$129,313",32,7%,2%,16%,4%,42%,22%,13%,2%,40.70,-74.02
10005,7135,97048,"$124,670",29,3%,2.3%,8.3%,0.7%,47.8%,23.9%,12.9%,4.1%,40.705,-74.005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14893,6,265,None,65,100%,27%,73%,0%,0%,0%,0%,0%,42.5,-77.0
14894,1454,129,"$40,417",44,96.9%,11%,68%,7%,7%,7%,0%,0%,42.03,-76.76
14897,980,38,"$41,905",38,83.0%,15%,68%,8%,6%,3%,0%,0%,42.04,-77.79
14898,1734,30,"$38,472",33,88.1%,24.3%,58.7%,9.0%,5.4%,2.2%,0.0%,0.3%,42.07,-77.43


In [233]:
demog_info.index[:2]

Index(['10001', '10002'], dtype='object')

In [227]:
zip_demog_EV = demog_info.join(merged_EV_stations_df)

In [228]:
zip_demog_EV.columns

Index(['Population', 'Population Density', 'Median Household Income',
       'Median Age:', 'Car, truck, or van', 'Less than High School Diploma',
       'High School Graduate', 'Associate's degree', 'Bachelor's degree',
       'Master's degree', 'Professional school degree', 'Doctorate degree',
       'Latitude', 'Longitude', 'reg_vehicles', 'num_stations'],
      dtype='object')

In [230]:
zip_demog_EV.dropna(subset=['Population', 'Population Density', 'Median Household Income',
       'Median Age:', 'Car, truck, or van', 'Less than High School Diploma',
       'High School Graduate', 'Associate\'s degree', 'Bachelor\'s degree',
       'Master\'s degree', 'Professional school degree', 'Doctorate degree',
       'Latitude', 'Longitude']).fillna(0).to_csv('Figures/zip_demog_EV.csv')